In [2]:
# https://www.nseindia.com/market-data/live-market-indices
import requests
import pandas as pd
from io import StringIO

# 1. Configuration
# CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_niftynext50list.csv"
# CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_nifty50list.csv"
# CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_niftybanklist.csv"
CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_nifty500list.csv"
# CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_niftymidcap50list.csv"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.nseindia.com/market-data/live-market-indices"
}

# 2. Warm up session to establish cookies
session = requests.Session()
session.headers.update(HEADERS)
session.get("https://www.nseindia.com", timeout=5)

# 3. Download the CSV
response = session.get(CSV_URL, timeout=10)
response.raise_for_status()

# 4. Parse into DataFrame
df_next50 = pd.read_csv(StringIO(response.text))
# print(df_next50.head())

# 5. Extract and print symbols and company names
symbols = df_next50["Symbol"].tolist()
companies = df_next50["Company Name"].tolist()

# Append ".NS" to each symbol
modified_symbols = [symbol + ".NS" for symbol in symbols]
print("NIFTY Next 50 Symbols:")
print(modified_symbols)

print("\nNIFTY Next 50 Companies:")
print(companies)


NIFTY Next 50 Symbols:
['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AWL.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ACE.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'APLLTD.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'ARE&M.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAZEN.NS', 'ASTRAL.NS', 'ATUL.NS', 'AUROPHARMA.NS', 'AIIL.NS', 'DMART.NS', 'AXISBANK.NS', 'BASF.NS', 'BEML.NS', 'BLS.NS', 'BSE.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKI